### Oracle AI Vector Search wrapped as a llama-index custom Vector Store

* inspired by: https://docs.llamaindex.ai/en/stable/examples/low_level/vector_store.html
* updated for **Llamaindex 0.10+**

In this **first demo** we show:
* how to embed a Text using OCI GenAI Embeddings (Cohere V3)
* How to query the Oracle AI Vector Store
* How to create a simplified QA retriever using LlamaIndex
* How to answer the question using a chatbot based on a LLM

In [1]:
import logging


from llama_index.core.vector_stores.types import (
    VectorStore,
    VectorStoreQuery,
    VectorStoreQueryResult,
)
from llama_index.core import VectorStoreIndex
from llama_index.core import Settings

import ads
from oci_utils import load_oci_config
from ads.llm import GenerativeAIEmbeddings, GenerativeAI
from oracle_vector_db import OracleVectorStore

from prepare_chain_4_chat import create_chat_engine

from config import EMBED_MODEL, TOP_K, TOP_N

from config_private import COMPARTMENT_OCID

/Users/lsaetta/miniforge3/envs/rag_env02/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# this is the endpoint after GA, for now Chicago Region
ENDPOINT = "https://inference.generativeai.us-chicago-1.oci.oraclecloud.com"

In [3]:
# Configure logging
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)

In [4]:
# setup for security
# in this example you need to have your api key in $HOME/.oci
# inside Data Science you can setup Resource Principal
oci_config = load_oci_config()

api_keys_config = ads.auth.api_keys(oci_config)

# english, or for other language use: multilingual

embed_model = GenerativeAIEmbeddings(
    compartment_id=COMPARTMENT_OCID,
    model=EMBED_MODEL,
    auth=api_keys_config,
    truncate="END",
    # Optionally you can specify keyword arguments for the OCI client, e.g. service_endpoint.
    client_kwargs={"service_endpoint": ENDPOINT},
)

# instantiate the client for the LLM
llm_oci = GenerativeAI(
    compartment_id=COMPARTMENT_OCID,
    # you can get these params also from config
    max_tokens=1024,
    temperature=0.1,
    client_kwargs={"service_endpoint": ENDPOINT},
)

#### Using the wrapper for AI Vector Sesarch to find documents relevant to the query

In [5]:
v_store = OracleVectorStore(verbose=True)

In [6]:
question = (
    # "What is JSON Relational Duality in Oracle Database 23c? Explain with details"
    "What is Oracle Data Guard? Can it be used for Disaster Recovery"
)

In [7]:
# embed the query using OCI GenAI

query_embedding = embed_model.embed_documents([question])[0]

#  wrap in llama-index
query_obj = VectorStoreQuery(query_embedding=query_embedding, similarity_top_k=TOP_K)

In [8]:
%%time

q_result = v_store.query(query_obj)

CPU times: user 20.5 ms, sys: 5.89 ms, total: 26.4 ms
Wall time: 728 ms


#### Displays results from AI Vector Search

In [9]:
for n, id, sim in zip(q_result.nodes, q_result.ids, q_result.similarities):

    print(n.text)
    print("")
    print(n.metadata)
    print(f"Similarity: {round(sim, 4)}")
    print("-------------------------------")

Part IV Oracle Data Guard Best Practices •Overview of MAA Best Practices for Oracle Data Guard •Plan an Oracle Data Guard Deployment •Configure and Deploy Oracle Data Guard •Tune and Troubleshoot Oracle Data Guard •Monitor an Oracle Data Guard Configuration 

{'file_name': 'high-availability-23c.pdf', 'page_label': '1'}
Similarity: 0.3112
-------------------------------
Part II Oracle Database High Availability Best Practices •Overview of Oracle Database High Availability Best Practices •Oracle Database Configuration Best Practices •Oracle Flashback Best Practices 

{'file_name': 'high-availability-23c.pdf', 'page_label': '1'}
Similarity: 0.3305
-------------------------------
7 High Availability Data Guard Oracle Data Guard Redo Decryption for Hybrid Disaster Recovery Configurations Oracle Data Guard now provides the capability to decrypt redo operations in hybrid cloud disaster recovery configurations where the cloud database is encrypted with Transparent Data Encryption (TDE) and th

#### Integrate in the RAG picture (the entire chain: Embeddings -> Vector Store -> LLM)

In [10]:
# replaced ServiceContext with Settings (llamaindex > 0.10)
Settings.embed_model = embed_model
Settings.llm = llm_oci

index = VectorStoreIndex.from_vector_store(vector_store=v_store)

query_engine = index.as_query_engine(similarity_top_k=TOP_K)

In [11]:
%%time

response = query_engine.query(question)

print(f"Question: {question}")
print("")
print(response.response)
print("")

Question: What is Oracle Data Guard? Can it be used for Disaster Recovery

Oracle Data Guard is a tool for database administrators to ensure data protection, high availability, and disaster recovery for enterprise data. It creates, maintains, manages, and monitors one or more standby databases to enable Oracle databases to survive outages of any kind, including natural disasters and data corruptions. It is a comprehensive set of services that can help protect your data and keep your database highly available. 

Yes, Oracle Data Guard is a recommended method for disaster recovery (DR). It provides a number of benefits for DR, including:

1. Hybrid disaster recovery: Oracle Data Guard supports hybrid disaster recovery in hybrid cloud environments. This allows you to expand outage and data protection to take advantage of the automation and resources of Oracle Cloud Infrastructure (OCI).
2. Encryption: Oracle Data Guard can now decrypt redo operations in hybrid cloud disaster recovery conf

#### Using the chat engine: a single line of code

In [12]:
chat_engine, token_counter = create_chat_engine(token_counter=None, verbose=False)

In [13]:
response = chat_engine.chat(question)

In [14]:
print(response)

Oracle Data Guard is a high availability feature used in Maximum Availability Architecture (MAA) solutions. It ensures availability and protection for enterprise data in the event of various outages, including natural disasters and data corruptions. 

Data Guard creates exact replicas, called standby databases, of primary databases, enabling efficient disaster recovery and data protection. The standby databases receive redo data from the primary database, maintaining synchronization and minimizing data loss. In the event of an outage, applications can switch to the standby database, minimizing downtime. 

Data Guard offers several benefits, such as continuous validation of changes, automatic corruption repair, reduced downtime for maintenance, and integration with other Oracle technologies. It also supports hybrid disaster recovery, allowing users to expand outage protection to cloud environments, such as Oracle Cloud Infrastructure (OCI). 

Yes, Oracle Data Guard can be effectively em

In [15]:
# if you want to count the token
token_counter.completion_llm_token_count

278

#### The chain keeps the conversation

In [16]:
question2 = "Can it be used to implement RPO close to zero?"

response2 = chat_engine.chat(question2)

In [17]:
print(response2)

Yes, Oracle Data Guard can help achieve a Recovery Point Objective (RPO) close to zero. An RPO represents the amount of data loss that is acceptable during an outage or disaster. With certain configurations and under specific conditions, Data Guard can significantly reduce data loss, thereby approaching zero RPO. 

To achieve near-zero RPO, Oracle Data Guard offers Maximum Availability Protection mode. In this mode, the standby database remains synchronous with the primary database, ensuring that it receives redo data synchronously. This means that the standby database is always current and can quickly take over in case of a failure, leading to minimal data loss. 

Additionally, Data Guard also provides the Fast Start Failover (FSFO) feature, which can further minimize RPO. FSFO enables the configuration of multiple observers and failover targets. These observers continuously monitor the health of the primary database, and upon detecting a failure, they initiate a failover to the desig